In [1]:
import pandas as pd

In [7]:
df = pd.read_parquet('../../data/update_data/demo1/realestate_dataset_hcm_v0.parquet')
df

,target,event_timestamp,realestate_id,typeOfRealEstate,accessibility,ward,certificateOfLandUseRight,houseDirection,street,district,facade,lon,frontWidth,lat,w,numberOfBathRooms,landSize,numberOfBedRooms,num_of_atm_in_1000m_radius
0,38.796680,2024-06-21 06:19:50+00:00,0,1,0,100,100,100,100,0,0.837641,106.733101,2.720505,10.876302,2.720505,4.477301,241.0,10.0,10.037441
1,135.135135,2024-06-21 06:22:48+00:00,1,1,0,100,100,100,100,7,0.546092,105.836563,2.969953,21.017610,2.969953,4.193552,37.0,1.0,49.000000
2,162.500000,2024-06-21 06:26:38+00:00,2,1,0,100,100,100,100,27,0.546092,105.817498,2.969953,21.029001,2.969953,4.000000,36.0,4.0,71.000000
3,99.500000,2024-06-21 06:34:35+00:00,3,1,0,100,100,100,100,13,0.837641,106.638767,10.000000,10.863220,10.000000,4.477301,20.0,2.0,10.037441
4,139.285714,2024-06-21 06:42:58+00:00,4,1,4,100,100,100,100,6,0.546092,105.886118,6.500000,21.029128,6.500000,4.000000,140.0,5.0,23.752708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,152.500000,2024-06-22 13:17:13+00:00,504,1,0,100,100,100,100,27,1.000000,105.810345,2.969953,21.029744,2.969953,4.193552,40.0,4.0,58.000000
505,280.000000,2024-06-22 13:17:13+00:00,505,1,0,100,100,100,100,6,2.000000,105.886118,2.969953,21.029128,2.969953,4.193552,100.0,4.0,23.752708
506,43.421053,2024-06-22 13:23:08+00:00,506,1,0,100,100,100,100,13,0.837641,106.677193,2.720505,10.854052,2.720505,4.477301,76.0,2.0,7.000000
507,54.883721,2024-06-22 13:26:39+00:00,507,0,0,100,100,2,100,2,0.546092,105.765531,2.969953,20.989024,2.969953,1.000000,43.0,1.0,23.752708


In [3]:
cat_cols = [
        "typeOfRealEstate",
        "accessibility",
        "ward",
        "certificateOfLandUseRight",
        "houseDirection",
        "street",
        "district"
    ]

In [4]:
df[cat_cols]

,typeOfRealEstate,accessibility,ward,certificateOfLandUseRight,houseDirection,street,district
0,1,0,100,100,100,100,17
1,1,0,100,100,100,100,34
2,1,0,100,100,100,100,5
3,1,0,100,100,100,100,13
4,1,4,100,100,100,100,5
...,...,...,...,...,...,...,...
995,1,0,100,100,100,100,17
996,1,0,100,100,100,100,27
997,1,0,100,100,100,100,19
998,1,0,100,100,100,100,19
